In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [ ]:
import sys
# sys.path.append('../')
sys.path.append('../')
sys.path.append('../model')
sys.path.append('../cost')
sys.path.append('../constraints')
sys.path.append('../utils')
import Aircraft3dofModel_tmp
import FinaltimeFreeCost
import Aircraft3dofConstraints
from scipy.integrate import solve_ivp
from Scvx_tf_free import Scvx_tf_free
from Scaling import TrajectoryScaling

In [ ]:
ix = 6
iu = 3
tf = 1000
N = 50
delT = tf/N
max_iter = 50

In [ ]:
myModel = Aircraft3dofModel_tmp.Aircraft3dof_tmp('Hello',ix,iu,linearization="numeric_central")
myCost = FinaltimeFreeCost.Finaltime('Hello',ix,iu,N)
myConst = Aircraft3dofConstraints.Aircraft3dof('Hello',ix,iu)


In [ ]:
xi = np.array([0*1e3,0*1e3,10*1e3,240,np.deg2rad(5),np.deg2rad(5)])

u0 = np.zeros((N+1,iu))
ui = np.array([0.0,np.deg2rad(0),1e3])
uf = np.array([0.0,-np.deg2rad(0),0])
for i in range(N+1) :
    u0[i] = (N-i)/N * ui + i/N * uf
for i in range(0,27) :
    u0[i,0] = (26-i)/26 * 0.3 + i/26 * 0.15
for i in range(27,N+1) :
    u0[i,0] = 1/700 * (i-27)**2 + 0.15

In [ ]:
def forward_full(x0,u,delT,model,type_discretization="foh") :
    def dfdt(t,x,um,up) :
        if type_discretization == "zoh" :
            u = um
        elif type_discretization == "foh" :
            alpha = (delT - t) / delT
            beta = t / delT
            u = alpha * um + beta * up
        return np.squeeze(model.forward(x,u,discrete=False))

    xnew = np.zeros((N+1,ix))
    xnew[0] = x0

    for i in range(N) :
        # sol = solve_ivp(dfdt,(0,self.delT),xnew[i],args=(u[i],u[i+1]),method='RK45',rtol=1e-6,atol=1e-10)
        sol = solve_ivp(dfdt,(0,delT),xnew[i],args=(u[i],u[i+1]))
        xnew[i+1] = sol.y[:,-1]

    return xnew,np.copy(u)

In [ ]:
myModel.set_scale(1,1,1,1,1)
x,u = forward_full(xi,u0,delT,myModel,"foh")

In [ ]:
# nondimension case
scl_t = 1e3
scl_x = 1e3
scl_f = 1e6
scl_kg = scl_f * scl_t * scl_t / scl_x
scl_v = scl_x/scl_t
scl_a = scl_x/scl_t**2
# scl_f = scl_kg * scl_x /scl_t/scl_t
print(scl_v,scl_a,scl_f)
# state and input
xi_ = np.array([xi[0],xi[1],xi[2]/scl_x,xi[3]/scl_v,xi[4],xi[5]])
u0_ = np.copy(u0)
u0_[:,2] = u0_[:,2] / scl_f
scl_rho = scl_kg / scl_x**3
scl_Sw = scl_x**2
print(scl_rho,scl_Sw)
# Scale model & constraints
myModel.set_scale(scl_x,scl_kg,scl_rho,scl_Sw,scl_a)
myConst.set_scale(scl_v,scl_f)
# time
tf_scaled = tf/scl_t

In [ ]:
xbar,ubar = forward_full(xi_,u0_,tf_scaled/N,myModel,"foh")

In [ ]:
xbar[:,0:3] = xbar[:,0:3] * scl_x
xbar[:,3] = xbar[:,3] * scl_v
ubar[:,2] = ubar[:,2] * scl_f

In [ ]:
print(np.max(u-ubar))

In [ ]:
np.max(np.abs(x-xbar))

In [ ]:
fig = plt.figure(1,figsize=(15,7))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.plot(x[:, 0], x[:, 1], x[:, 2],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 1], xbar[:, 2],label='solution')
plt.legend()
fig = plt.figure(2,figsize=(15,5))
ax = fig.add_subplot(131)
ax.plot(x[:, 0], x[:, 1],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 1],label='solution')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.legend()
ax = fig.add_subplot(132)
ax.plot(x[:, 1], x[:, 2],label='single shooting')
ax.plot(xbar[:, 1], xbar[:, 2],label='solution')
ax.set_xlabel('Y')
ax.set_ylabel('Z')
ax = fig.add_subplot(133)
ax.plot(x[:, 0], x[:, 2],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 2],label='solution')
ax.set_xlabel('X')
ax.set_ylabel('Z')
plt.legend()

In [ ]:
# state
xi = np.array([0,0,10*1e3/scl_x,240/scl_v,0,0])
xf = np.array([110*1e3/scl_x,-60*1e3/scl_x,0,95/scl_v,-np.deg2rad(3),np.deg2rad(80)])
print(xi)
print(xf)
x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf
# input    
u0 = np.zeros((N+1,iu))
ui = np.array([0.5,np.deg2rad(0),0])
uf = np.array([0.5,-np.deg2rad(0),0])
for i in range(N+1) :
    u0[i] = (N-i)/N * ui + i/N * uf
# for i in range(0,27) :
#     u0[i,0] = (26-i)/26 * 0.3 + i/26 * 0.15
# for i in range(27,N+1) :
#     u0[i,0] = 1/700 * (i-27)**2 + 0.15
u0[:,2] = u0[:,2] / scl_f

In [ ]:
x_min = np.zeros(ix)
x_max = np.array([110*1e3/scl_x,60*1e3/scl_x,10*1e3/scl_x,240/scl_v,np.deg2rad(20),np.pi/2])
print(x_max)
u_min = np.array([0,0,0])
u_max = np.array([1.52,np.deg2rad(15),1126.3 * 1e3/scl_f]) 
print(u_max)

# x_max = np.ones(ix)
# u_max = np.ones(iu)

myScaling = TrajectoryScaling(x_min,x_max,u_min,u_max,1)

In [ ]:
i1 = Scvx_tf_free('aircraft',N,tf_scaled,max_iter,myModel,myCost,myConst,myScaling,
          type_discretization='foh',w_c=1,w_vc=1e-2*6,w_tr=1e-2*2,tol_vc=1e-10,tol_tr=1e-10,tol_bc=1e-3,flag_policyopt=False)
x,u,xbar,ubar,tfbar,total_num_iter,flag_boundary,l,l_vc,l_tr,x_traj,u_traj = i1.run(x0,u0,xi,xf)

In [ ]:
tfbar = tfbar*scl_t
print(tfbar)
xbar[:,0:3] = xbar[:,0:3] * scl_x
xbar[:,3] = xbar[:,3] * scl_v

x[:,0:3] = x[:,0:3] * scl_x
x[:,3] = x[:,3] * scl_v

ubar[:,2] = ubar[:,2] * scl_f
u[:,2] = u[:,2] * scl_f

In [ ]:
fig = plt.figure(1,figsize=(15,7))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.plot(x[:, 0], x[:, 1], x[:, 2],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 1], xbar[:, 2],label='solution')
plt.legend()
fig = plt.figure(2,figsize=(15,5))
ax = fig.add_subplot(131)
ax.plot(x[:, 0], x[:, 1],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 1],label='solution')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.legend()
ax = fig.add_subplot(132)
ax.plot(x[:, 1], x[:, 2],label='single shooting')
ax.plot(xbar[:, 1], xbar[:, 2],label='solution')
ax.set_xlabel('Y')
ax.set_ylabel('Z')
ax = fig.add_subplot(133)
ax.plot(x[:, 0], x[:, 2],label='single shooting')
ax.plot(xbar[:, 0], xbar[:, 2],label='solution')
ax.set_xlabel('X')
ax.set_ylabel('Z')
plt.legend()
# ax.plot(xbar[:, 0], xbar[:, 1], xbar[:, 2],'o')

In [ ]:
t_index = np.array([i for i in range(N+1)]) / N * tfbar
plt.figure(figsize=(15,15))
plt.subplot(331)
plt.plot(t_index,xbar[:,0],label='solution')
plt.plot(t_index,x[:,0],'--',label='single shooting')
plt.legend(fontsize=15)
plt.title('x (m)')

plt.subplot(332)
plt.plot(t_index,xbar[:,1])
plt.plot(t_index,x[:,1],'--')
plt.title('y (m)')

plt.subplot(333)
plt.plot(t_index,xbar[:,2])
plt.plot(t_index,x[:,2],'--')
plt.title('z (m)')


plt.subplot(334)
plt.plot(t_index,xbar[:,3])
plt.plot(t_index,x[:,3],'--')
plt.plot(t_index,ubar[:,2]*0 + myConst.v_max,'--',color='tab:red')
plt.plot(t_index,ubar[:,2]*0 + myConst.v_min,'--',color='tab:red')
plt.title('velocity (m/s)')

plt.subplot(335)
plt.plot(t_index,xbar[:,4])
plt.plot(t_index,x[:,4],'--')
plt.plot(t_index,ubar[:,2]*0 + myConst.gamma_max,'--',color='tab:red')
plt.plot(t_index,ubar[:,2]*0 + myConst.gamma_min,'--',color='tab:red')
plt.title('gamma (rad)')

plt.subplot(336)
plt.plot(t_index,xbar[:,5])
plt.plot(t_index,x[:,5],'--')
plt.title('psi (rad)')

plt.subplot(337)
plt.plot(t_index,ubar[:,0])
# plt.plot(t_index,ubar[:,2]*0 + myConst.CL_max,'--',color='tab:red')
# plt.plot(t_index,ubar[:,2]*0 + myConst.CL_min,'--',color='tab:red')
plt.title('CL')
plt.plot(t_index,u0[:,0],'--',label='initial')
plt.legend(fontsize=15)
plt.subplot(338)
plt.plot(t_index,ubar[:,1])
plt.plot(t_index,ubar[:,2]*0 + myConst.phi_max,'--',color='tab:red')
plt.plot(t_index,ubar[:,2]*0 + myConst.phi_min,'--',color='tab:red')
plt.title('phi (rad)')
plt.plot(t_index,u0[:,1],'--',label='initial')
plt.subplot(339)
plt.plot(t_index,ubar[:,2])
plt.plot(t_index,ubar[:,2]*0 + myConst.T_max,'--',color='tab:red')
plt.plot(t_index,ubar[:,2]*0 + myConst.T_min,'--',color='tab:red')
plt.title('thrust (N)')
plt.plot(t_index,u0[:,2]*scl_f,'--',label='initial')

In [ ]:
for xbar_ in x_traj :
    plt.plot(t_index,xbar_[:,2],alpha=0.5,color='tab:blue')
# plt.title('CL')

In [ ]:
# for xbar_ in x_traj :
plt.plot(t_index,x_traj[0][:,2],alpha=0.5,color='tab:blue')
# plt.title('CL')

In [ ]:
T = 15.04 - 0.00649 * xbar[:,2] # celsius
p = 101.29 * np.power((T+273.1)/288.08,5.256)
rho = p / (0.2869 * (T + 273.1))

F_lift = 0.5 * rho * xbar[:,3] * xbar[:,3] * myModel.Sw * ubar[:,0]
F_draf = 0.5 * rho * xbar[:,3] * xbar[:,3] * myModel.Sw * (myModel.CD0 + myModel.K  * ubar[:,0] * ubar[:,0])

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(t_index,rho)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(338)
plt.plot(t_index,F_lift)
plt.title('Lift force (N)')

plt.subplot(339)
plt.plot(t_index,F_draf)
plt.title('Drag force (N)')

In [ ]:
for ubar_ in u_traj :
    plt.plot(t_index,ubar_[:,2],alpha=0.5,color='tab:blue')
# plt.title('CL')

In [ ]:
for ubar_ in u_traj[48:] :
    plt.plot(t_index,ubar_[:,2],alpha=0.5,color='tab:blue')
# plt.title('CL')